In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

It is kind of boring to be using MNIST, but it is an excellent starting point for a tutorial...

In [2]:
dataset = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = dataset.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Adding channel dimension for a convolutional layer
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


We use the subclassing API to build our model

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

We define our loss function and optimizer ...

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()


And, our training/test metrics

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


We define our own special training step for full flexibility in the training loop.

**Note the @tf.function which tells Tensorflow to compile a finished graph of the underlying functions.** 

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


We can do the same to the test step

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


Finally, let us train this model. 

**Note that we test the model over the full test set each epoch. This is strictly not necessary...**

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.13276052474975586, Accuracy: 96.01000213623047, Test Loss: 0.0694454237818718, Test Accuracy: 97.65999603271484
Epoch 2, Loss: 0.04167613759636879, Accuracy: 98.69499969482422, Test Loss: 0.05247476324439049, Test Accuracy: 98.27999877929688
Epoch 3, Loss: 0.021755658090114594, Accuracy: 99.30166625976562, Test Loss: 0.05823026970028877, Test Accuracy: 98.19999694824219
Epoch 4, Loss: 0.013308051973581314, Accuracy: 99.55166625976562, Test Loss: 0.05713808909058571, Test Accuracy: 98.38

### Finished?

* [ ] Make your own customized Layer by subclassing and integrate it into your model
* [ ] Now that you are an TF2 expert, I recommend you to look into the following tutorials:
    * **Structured data:** https://www.tensorflow.org/tutorials/structured_data/feature_columns
    * **Text classification:** https://www.tensorflow.org/tutorials/text/text_classification_rnn
    * **Time series:** https://www.tensorflow.org/tutorials/structured_data/time_series
    * **GAN:** https://www.tensorflow.org/tutorials/generative/dcgan

If you are sick of tutorials and want more challenges. 
Try to build and train a model for another domain (not image classification) without looking at the documentation.